# Facenet repo


```
https://github.com/davidsandberg/facenet
```



In [ ]:
# Insightface repo


```
https://github.com/deepinsight/insightface.git
```

# Google Drive Directory


```
https://drive.google.com/drive/folders/1vqKryG2EC2PyHxdvjzEyou6eaqIZu5Rq?usp=sharing
```



# Preparing data

In [0]:
# cloning facenet repo
!git clone https://github.com/davidsandberg/facenet

# downloading VGGFace2 pretrained model
# https://drive.google.com/file/d/1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-/view
!cp '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/20180402-114759.zip' /content/

#unpacking the model
!mkdir /content/facenet/model/
!unzip /content/20180402-114759.zip -d /content/facenet/model/

#clean up
!rm /content/20180402-114759.zip

# **Face Detection & Alignment**
  using insightface mxnet model

Prepare needed files

In [0]:
# cloning insightface repo
!pip install mxnet-cu100
!git clone --recursive https://github.com/deepinsight/insightface.git

In [0]:
# downloading pretrained detection model
!wget https://www.dropbox.com/s/53ftnlarhyrpkg2/retinaface-R50.zip

# unpacking the model
!unzip /content/retinaface-R50.zip -d /content/insightface/models/

# unpacking cython files
!cd /content/insightface/RetinaFace/rcnn/cython ;  python setup.py build_ext --inplace

loading data and needed scripts

In [0]:
#copying our test data from drive
!cp -r /content/drive/My\ Drive/Motion\ Capture\ project\ ITI\ intake40/Data/Faces\ images/unprocessed /content/data

#load modified scripts
!cp '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/align_dataset_mtcnn.py' /content/facenet/src/align/ 

Execute script

In [0]:
!python /content/facenet/src/align/align_dataset_mtcnn.py --image_size 160 --margin 20 --detect_multiple_faces True /content/data/ /content/aligned/
# input dir /content/data/    output dir /content/aligned/

save new data and scripts

In [0]:
!cp /content/facenet/src/align/align_dataset_mtcnn.py '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/'

# copy aligned folder to drive to manually clean
!cp -r /content/aligned/  '/content/drive/My Drive/Motion Capture project ITI intake40/Data/Faces images/processed'

# **Data augmentation**

In [0]:
#copying our test data from drive
!cp -r /content/drive/My\ Drive/Motion\ Capture\ project\ ITI\ intake40/Data/Faces\ images/processed /content/data

In [0]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null && echo "All libraries are successfully installed!"

import numpy as np
import cv2
from matplotlib import pyplot as plt
from IPython.display import display, HTML 

from albumentations import (VerticalFlip, HorizontalFlip, Flip, RandomRotate90, Rotate, ShiftScaleRotate, CenterCrop, OpticalDistortion, GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
                            RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise, CLAHE, ChannelShuffle, InvertImg, RandomGamma, ToGray, PadIfNeeded 
                           )

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-1jy3fli2
All libraries are successfully installed!


helper functions

In [0]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [0]:
#@title Enter input data directory { run: "auto", vertical-output: true, display-mode: "form" }
from PIL import Image
import random
import os

# load a random image to show augmentation effects
def load_image(dir_path):
    path_exp = os.path.expanduser(dir_path)
    paths = [path for path in os.listdir(path_exp) if os.path.isdir(os.path.join(path_exp, path))]
    count = 0
    while True:
      if count == 10:
        return None
      file_name = ""
      try:
        directory = random.choice(paths)
        file_name = random.choice(os.listdir(dir_path + directory))
        img = Image.open(dir_path + directory +'/'+ file_name)
        return dir_path + directory +'/'+ file_name
      except:
        count = count + 1
        print('NOT IMAGE: ', dir_path + directory +'/'+ file_name)
        continue

data_path =  '/content/data/aligned/' #@param {type:"string"}
path = load_image(data_path)
print('test image: ', path)
if path != None:
  show_img(Image.open(path))

In [0]:
#@title Choose required augmentations { run: "auto", vertical-output: true, display-mode: "form" }
def augment_image(img_path, show_output=True):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if len(img) > 0:
        angle = 96 #@param {type:"slider", min:0, max:360, step:1}
        shift = 0 #@param {type:"slider", min:0, max:1, step:0.05}
        scale = 1 #@param {type:"slider", min:0, max:2, step:0.1}

        aug = ShiftScaleRotate(p=0.5)
        img = aug.apply(img, angle=angle, scale=scale, dx=shift, dy=shift)

        flip_image = "Disabled" #@param ["Vertical", "Horizontal", "Both Horizontal and Vertical", "Disabled"]
        state_to_code = {
            "Both Horizontal and Vertical": -1,
            "Vertical": 0,
            "Horizontal": 1,
        }
        aug = Flip(p=0.5)
        if flip_image == "Disabled":
          pass
        else:
          img = aug.apply(img, d=state_to_code[state]) 
        #@markdown ---
        Brightness = False #@param {type:"boolean"}
        if Brightness:
            alpha = 0.6 #@param {type:"slider", min:0, max:5, step:0.1}
            aug = RandomBrightness(p=0.5)
            img = aug.apply(img, alpha=alpha)

        Contrast = False #@param {type:"boolean"}
        if Contrast:
          alpha = 0.9 #@param {type:"slider", min:0, max:3, step:0.1}
          aug = RandomContrast(p=0.5)
          img = aug.apply(img, alpha=alpha)
        #@markdown ---
        RGB_Shift = False #@param {type:"boolean"}
        if RGB_Shift:
          r_shift = 0 #@param {type:"slider", min:-255, max:255, step:1}
          g_shift = 45 #@param {type:"slider", min:-255, max:255, step:1}
          b_shift = 40 #@param {type:"slider", min:-255, max:255, step:1}

          aug = RGBShift(p=0.5)
          img = aug.apply(img, r_shift=r_shift, g_shift=g_shift, b_shift=b_shift)

        hue_saturation = False #@param {type:"boolean"}
        if hue_saturation:
          hue_shift = 122 #@param {type:"slider", min:0, max:180, step:1}
          sat_shift = 97 #@param {type:"slider", min:0, max:255, step:1}
          val_shift = 45 #@param {type:"slider", min:0, max:255, step:1}

          aug = HueSaturationValue(p=0.5)
          img = aug.apply(img, hue_shift=hue_shift, sat_shift=sat_shift, val_shift=val_shift)

        gamma = False #@param {type:"boolean"}
        if gamma:
          gamma = 37 #@param {type:"slider", min:0, max:255, step:1}
          aug = RandomGamma(p=0.5)
          img = aug.apply(img, gamma=gamma / 100)

        #@markdown ---
        Elastic_transform = False #@param {type:"boolean"}
        if Elastic_transform:
          alpha = 203 #@param {type:"slider", min:0, max:255, step:1}
          alpha_affine = 106 #@param {type:"slider", min:0, max:255, step:1}
          sigma = 166 #@param {type:"slider", min:0, max:255, step:1}
          aug = ElasticTransform(p=0.5)
          img = aug.apply(img, alpha=alpha, sigma=sigma, alpha_affine=alpha_affine)
        
        #@markdown ---
        Optical_Distortion = False #@param {type:"boolean"}
        if Optical_Distortion:
          distort_limit = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
          shift_limit = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
          aug = OpticalDistortion(p=0.5)
          img = aug.apply(img, k=distort_limit, dx=shift_limit, dy=shift_limit)
        
        #@markdown ---
        Center_crop = False #@param {type:"boolean"}
        if Center_crop:
          size = 288 #@param {type:"slider", min:32, max:512, step:16}
          aug = CenterCrop(height=size, width=size, p=0.5)
          img = aug.apply(img)
        
        #@markdown ---
        jpeg_compression = False #@param {type:"boolean"}
        if jpeg_compression:
          quality = 23 #@param {type:"slider", min:0, max:100, step:1}
          aug = JpegCompression(p=1)
          img = aug.apply(img, quality=quality)

        if show_output:
          show_img(img)
        return img

ret = augment_image(path)

In [0]:
# apply to all images
output_directory = '/content/augmented/'
path_exp = os.path.expanduser(data_path)
paths = [path for path in os.listdir(path_exp) if os.path.isdir(os.path.join(path_exp, path))]
for directory in paths:
    for file in os.listdir(path_exp + directory):
        if os.path.isdir(path_exp + directory + '/' + file):
          paths.append(path_exp + directory + '/' + file)
        else:
          ret = augment_image(path_exp + directory + '/' + file, show_output=False)
          if not os.path.exists(output_directory + directory):
              os.makedirs(output_directory + directory)
          cv2.imwrite(output_directory + directory + '/' + file, ret)

# **Face Recognition**

Preparing test specific data

In [0]:
#copying our test data from drive
!cp -r /content/drive/My\ Drive/Motion\ Capture\ project\ ITI\ intake40/Data/Faces\ images/processed /content/data

#load modified scripts
!cp '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/classifier.py' /content/facenet/src/ 
!cp '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/facenet.py' /content/facenet/src/

In [0]:
#running test script
!python /content/facenet/src/classifier.py /content/data/aligned/ /content/facenet/model/20180402-114759/ /content/aligned/zeinab/10626407_1039483162746713_2617121640081663520_o_2.png
# data_dir '/content/data/train/' model '/content/facenet/model/20180402-114759/'

In [0]:
from PIL import Image
import matplotlib.pyplot as plt

img = Image.open('/content/aligned/zeinab/10626407_1039483162746713_2617121640081663520_o_2.png')
plt.imshow(img)

Save modified scripts

In [0]:
!cp /content/facenet/src/classifier.py '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/'
!cp /content/facenet/src/facenet.py  '/content/drive/My Drive/Motion Capture project ITI intake40/scripts/Facenet/'